# Intro  (contexto)

## Contexto

## Objetivo


## Breve descripción

In [115]:
# importamos todas las librerias necesarios

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [117]:
# Preparo el driver

ser=Service('Documents/Programación/Web scrapping/Selenium/chromedriver.exe')
driver = webdriver.Chrome(service=ser)

In [118]:
# Abro la pagina principal del CRM

url = "https://login.CRMCRMCRM.org/"

driver.get(url)
time.sleep(2)


In [119]:
#INICIO SESIÓN INCLUIDO CODIGO DE SEGURIDAD


#cargo las credenciales
user= input()      #configurar interfaz del input
passw= input()     #configurar interfaz del input    


#ingreso las credenciales en los campos
driver.find_element(by=By.XPATH,value='//INPUT[@name="email"]').send_keys(user)
driver.find_element(by=By.XPATH,value='//INPUT[@name="password"]').send_keys(passw)

driver.find_element(by=By.XPATH,value='//*[@id="app"]/div[3]/section/div[2]/div/div/div/div[4]/div[2]/button').click()
time.sleep(2)

#ya en la siguiente pantalla realizo el envío de 
boton_sendcode = driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div[2]/div/div/div/div/div[3]/div[2]/button')
boton_sendcode.click()

sec_code=input()

for i in range(1,7):
    input_code= driver.find_element(by=By.XPATH,value='//*[@id="app"]/div[3]/section/div[2]/div/div/div/div/div[3]/div/div/div['+str(i)+']/input')
    input_code.send_keys(sec_code[i-1])

confirm_code = driver.find_element(by=By.XPATH,value='//*[@id="app"]/div[3]/section/div[2]/div/div/div/div/div[4]/div[2]/button')
confirm_code.click()

 419868


A esta altura ya estamos logueados en el CRM en la página /angecy_dashboard








In [ ]:
#añado la columna "Ad" y filtro por leads en lead transferred

def prep_list():
    driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/button').click()  #abro el selector de columnas
    time.sleep(3)
    driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/button').click()  #abro el selector de columnas
    time.sleep(3)
    driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/ul/li[7]/div/input').click()  #marco la columna Ad
    time.sleep(2)
    
    #filtro por pipeline stage Lead Transferred
    driver.find_element(by=By.XPATH,value='//*[@id="hl_smartlists--filter-btn"]').click()  #abro el selector de filtros
    time.sleep(2)
    driver.find_element(by=By.XPATH,value='//*[@id="SmartListFilter-Opportunity_Information-Pipeline_Stage"]').click()  #selecciono pipeline stage
    time.sleep(2)
    driver.find_element(by=By.XPATH,value='//*[@id="dropdownMenuButton1"]').click()  #despliego para seleccionar la pipelinetime.sleep(2)
    time.sleep(2)
    driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[4]/div[2]/div/div[2]/div[4]/div/div/div/div/div[1]/div/div[1]').click()  #selecciono la primer pipe
    time.sleep(2)
    driver.find_element(by=By.XPATH,value='//*[@id="dropdownMenuButton2"]').click() #despliego para seleccionar el stage
    time.sleep(2)
    driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[4]/div[2]/div/div[2]/div[4]/div/div/div/div/div[2]/div/div[10]').click() #selecciono lead transferred
    time.sleep(2)
    driver.find_element(by=By.XPATH,value='//*[@id="smartlists"]/div[4]/div[2]/div/div[3]/div/button').click()  #aplico el filtro        
    time.sleep(2)


In [120]:
#para entrar a esta parte tienen que estar la pagina con el desplegable de las columnas abiertas y que sólo ya estén seleccionadas 6 columnas

def select_all_columns(driver):
    driver.find_element(by=By.XPATH,value='//*[@id="colViewButton"]').click()
    try:
        driver.find_element(by=By.XPATH,value='//*[@id="colViewButton"]').click()
    except:
        next
    columns=driver.find_elements(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/ul//li')
    for i in range(len(columns)-6):
        driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/ul/li['+str(len(columns))+']/div/input').click()
        time.sleep(0.2)

In [134]:
#FUNCION PARA DESCARGAR LA SMAT LIST YA ESTANDO FILTRADA Y CON LAS COLUMNAS SELECCIONADAS


def desc(driver):
    driver.find_element(by=By.XPATH,value='//*[@id="all"]').click()    # seleccionar la lista
    time.sleep(3)

    try:    
        driver.find_element(by=By.XPATH,value='//*[@id="hl_smartlists-main"]/div[1]/div/div[1]/div/span[2]').click()    
    except:
        next
        

    boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="smartlists"]/div[2]/div[1]/span[10]/button')))
    boton.click()    #boton para exportar ya estaado en la smartlist
    
    boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div[2]/div/div/div/button')))
    boton.click()    #boton para iniciar la descarga estando todo preparado
    
    time.sleep(1)
    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="all"]')))   #revisar esta linea


In [145]:
ALLS="6ZNDYU7FC7suAYtH81sC,vy3HXxVjRbBen4gDHTBT,335tvBVvpfEsc95omLWD"

ids=ALLS.split(",")

In [ ]:
ids

In [ ]:
# EJECUTO SECUENCIA PARA DESCARGAR LOS TRANSFERIDOS CON LA COLUMNA AD
#solo necesito estar logueado Y tener la lsita de los ids de las cuentas que quiero descargar
for i in ids:
    url="https://login.murraymedia.org/v2/location/"+i+"/contacts/smart_list/All"
    driver.get(url)
    time.sleep(10)
    prep_list()
    desc()


In [146]:
# EJECUTO SECUENCIA PARA DESCARGAR TODOS LOS CONTACTS CON TODAS LAS COLUMNAS
#solo necesito estar logueado Y tener la lista de los ids de las cuentas que quiero descargar
for i in ids:
    url="https://login.murraymedia.org/v2/location/"+i+"/contacts/smart_list/All"
    driver.get(url)
    time.sleep(10)
    select_all_columns(driver)  #esto sería preparar la lista full
    desc(driver)
    print(i)


6ZNDYU7FC7suAYtH81sC
vy3HXxVjRbBen4gDHTBT
335tvBVvpfEsc95omLWD


In [ ]:
url="https://login.murraymedia.org/v2/location/1OR0TrE51C0cICLHLUlp/contacts/smart_list/All"
driver.get(url)
time.sleep(10)

In [ ]:
driver.find_element(by=By.XPATH,value='//*[@id="colViewButton"]').click()
try:
    driver.find_element(by=By.XPATH,value='//*[@id="colViewButton"]').click()
except:
    next


In [84]:
##Actualizar custom field
url="https://login.murraymedia.org/v2/location/"+str(ids[0])+"/contacts/smart_list/All"
driver.get(url)
time.sleep(10)

# columns=driver.find_elements(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/ul//li')
# for i in range(len(columns)-6):
#     driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/section/div/section/div[2]/div[2]/div[1]/ul/li['+str(len(columns))+']/div/input').click()
#     time.sleep(0.2)

In [ ]:
boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'')))
boton.click()

In [114]:
#selecciono todos los contacts
driver.find_element(by=By.XPATH,value='//*[@id="all"]').click()    # seleccionar la lista
time.sleep(3)

try:    
    driver.find_element(by=By.XPATH,value='//*[@id="hl_smartlists-main"]/div[1]/div/div[1]/div/span[2]').click()    
except:
    next
      
boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="smartlists"]/div[2]/div[1]/span[3]/button')))
boton.click()  #click en addowrkflow

boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div/div/div[2]/div[2]/div/button')))
boton.click() #click en ok procedd

boxinput= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div/div/div[2]/div[1]/div/div[1]/div/div/div/div[1]/input')))
boxinput.send_keys('Workflow - Actualizar custom field')
boxinput.send_keys(Keys.ENTER)

boxinput= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div/div/div[2]/div[1]/div/div[4]/div/input')))
boxinput.send_keys('update')
time.sleep(1)
boxinput.send_keys(Keys.ENTER)

boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div/div/div[2]/div[2]/div/button')))
boton.click()

boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div/div/div[2]/div[2]/div/button')))
boton.click()

In [109]:
boxinput= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div/div/div[2]/div[1]/div/div[1]/div/div/div/div[1]/input')))
boxinput.send_keys('Workflow - Actualizar custom field')
boxinput.send_keys(Keys.ENTER)


In [112]:
boxinput= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div/div/div[2]/div[1]/div/div[4]/div/input')))
boxinput.send_keys('update')

In [ ]:
dir(boxinput)

In [100]:
boton= WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[7]/div[1]/div/div/div/div/div/div[2]/div[2]/div/button')))
boton.click()

In [133]:
dir(EC
)

['NoAlertPresentException',
 'NoSuchElementException',
 'NoSuchFrameException',
 'StaleElementReferenceException',
 'WebDriverException',
 'WebElement',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_element_if_visible',
 'alert_is_present',
 'all_of',
 'any_of',
 'element_attribute_to_include',
 'element_located_selection_state_to_be',
 'element_located_to_be_selected',
 'element_selection_state_to_be',
 'element_to_be_clickable',
 'element_to_be_selected',
 'frame_to_be_available_and_switch_to_it',
 'invisibility_of_element',
 'invisibility_of_element_located',
 'new_window_is_opened',
 'none_of',
 'number_of_windows_to_be',
 'presence_of_all_elements_located',
 'presence_of_element_located',
 're',
 'staleness_of',
 'text_to_be_present_in_element',
 'text_to_be_present_in_element_attribute',
 'text_to_be_present_in_element_value',
 'title_contains',
 'title_is',
 'url_changes',
 'url_contains',
 'url_matches',
 'u